In [57]:
import json
import dbconn
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.dialects.mysql import insert

In [58]:
def insert_on_duplicate(table, conn, keys, data_iter):
    insert_stmt = insert(table.table).values(list(data_iter))
    on_duplicate_key_stmt = insert_stmt.on_duplicate_key_update(insert_stmt.inserted)
    conn.execute(on_duplicate_key_stmt)

In [59]:
ikea_dataset = pd.read_csv('ikea.csv', index_col='item_id')
ikea_dataset = ikea_dataset.drop(columns=['Unnamed: 0', 'sellable_online', 'old_price'])

In [60]:
ikea_dataset = ikea_dataset.iloc[::4]

In [61]:
link_list = list(zip(ikea_dataset['link'],ikea_dataset['link'].index))

In [62]:
ikea_dataset

,name,category,price,link,other_colors,short_description,designer,depth,height,width
item_id,,,,,,,,,,
90420332,FREKVENS,Bar furniture,265.0,https://www.ikea.com/sa/en/p/frekvens-bar-tabl...,No,"Bar table, in/outdoor, 51x51 cm",Nicholai Wiig Hansen,NaN,99.0,51.0
30180504,NORBERG,Bar furniture,225.0,https://www.ikea.com/sa/en/p/norberg-wall-moun...,No,"Wall-mounted drop-leaf table, ...",Marcus Arvonen,60.0,43.0,74.0
50406465,FRANKLIN,Bar furniture,129.0,https://www.ikea.com/sa/en/p/franklin-bar-stoo...,No,"Bar stool with backrest, foldable, ...",K Hagberg/M Hagberg,44.0,95.0,50.0
397736,NORRARYD,Bar furniture,395.0,https://www.ikea.com/sa/en/p/norraryd-bar-stoo...,No,"Bar stool with backrest, 74 cm",Nike Karlsson,47.0,103.0,46.0
40426138,HENRIKSDAL,Bar furniture,395.0,https://www.ikea.com/sa/en/p/henriksdal-bar-st...,No,"Bar stool with backrest frame, ...",Karl Malmvall,51.0,102.0,40.0
...,...,...,...,...,...,...,...,...,...,...
59315159,ELVARLI,Wardrobes,2397.0,https://www.ikea.com/sa/en/p/elvarli-3-section...,No,"3 sections, 258x51x222-350 cm",Ehlén Johansson,51.0,NaN,258.0
19157897,ELVARLI,Wardrobes,1226.0,https://www.ikea.com/sa/en/p/elvarli-2-section...,No,"2 sections, 92/92x222-350 cm",Ehlén Johansson,NaN,NaN,91.0
49157565,ELVARLI,Wardrobes,1245.0,https://www.ikea.com/sa/en/p/elvarli-2-section...,No,"2 sections, 165x55x216 cm",Ehlén Johansson/IKEA of Sweden,55.0,216.0,164.0


In [63]:
link_list

[('https://www.ikea.com/sa/en/p/frekvens-bar-table-in-outdoor-black-90420332/',
  90420332),
 ('https://www.ikea.com/sa/en/p/norberg-wall-mounted-drop-leaf-table-white-30180504/',
  30180504),
 ('https://www.ikea.com/sa/en/p/franklin-bar-stool-with-backrest-foldable-black-black-50406465/',
  50406465),
 ('https://www.ikea.com/sa/en/p/norraryd-bar-stool-with-backrest-black-00397736/',
  397736),
 ('https://www.ikea.com/sa/en/p/henriksdal-bar-stool-with-backrest-frame-dark-brown-40426138/',
  40426138),
 ('https://www.ikea.com/sa/en/p/ekedalen-bar-table-white-00340759/', 340759),
 ('https://www.ikea.com/sa/en/p/ekedalen-bar-stool-with-backrest-white-orrsta-light-grey-00400545/',
  400545),
 ('https://www.ikea.com/sa/en/p/stensele-norraryd-bar-table-and-2-bar-stools-anthracite-anthracite-black-s29297227/',
  29297227),
 ('https://www.ikea.com/sa/en/p/norraker-norraker-bar-table-and-2-bar-stools-birch-birch-s19297275/',
  19297275),
 ('https://www.ikea.com/sa/en/p/nordviken-bar-stool-with-

In [64]:
import requests
from bs4 import BeautifulSoup

def get_url_href_results(URL):
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    return soup

def check_product_page_availability(search_term):
    return get_url_href_results(search_term)

You can stop this process whenever you feel like you have enough IKEA products to add to your database! It can take a long time to web scrape and verify all of the images from all of the IKEA products.

In [65]:
valid_product_id_list = []
valid_product_image_list = []

n = 0

for link, index in link_list:
    soup = check_product_page_availability(link)

    for div in soup.find_all(id="pip-media-1"):
        n += 1
        print("Product number: ", n)
        
        image_link = div.find('img')
        print("Found the Image:", image_link["src"])
        image_link = image_link["src"]
        
        valid_product_id_list.append(index)
        valid_product_image_list.append(image_link)
        
print((valid_product_id_list, valid_product_image_list))

Product number:  1
Found the Image: https://www.ikea.com/sa/en/images/products/norberg-wall-mounted-drop-leaf-table-white__0736622_pe740674_s5.jpg?f=s
Product number:  2
Found the Image: https://www.ikea.com/sa/en/images/products/franklin-bar-stool-with-backrest-foldable-black-black__0727485_pe735710_s5.jpg?f=s
Product number:  3
Found the Image: https://www.ikea.com/sa/en/images/products/norraryd-bar-stool-with-backrest-black__0728070_pe736043_s5.jpg?f=s
Product number:  4
Found the Image: https://www.ikea.com/sa/en/images/products/ekedalen-bar-table-white__0657888_pe710020_s5.jpg?f=s
Product number:  5
Found the Image: https://www.ikea.com/sa/en/images/products/ekedalen-bar-stool-with-backrest-white-orrsta-light-grey__0657871_pe710003_s5.jpg?f=s
Product number:  6
Found the Image: https://www.ikea.com/sa/en/images/products/stensele-norraryd-bar-table-and-2-bar-stools-anthracite-anthracite-black__0680383_pe719836_s5.jpg?f=s
Product number:  7
Found the Image: https://www.ikea.com/sa/e

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [87]:
valid_product_image_list = pd.Series(valid_product_image_list).drop_duplicates().to_list()

In [88]:
valid_product_image_list

['https://www.ikea.com/sa/en/images/products/norberg-wall-mounted-drop-leaf-table-white__0736622_pe740674_s5.jpg?f=s',
 'https://www.ikea.com/sa/en/images/products/franklin-bar-stool-with-backrest-foldable-black-black__0727485_pe735710_s5.jpg?f=s',
 'https://www.ikea.com/sa/en/images/products/norraryd-bar-stool-with-backrest-black__0728070_pe736043_s5.jpg?f=s',
 'https://www.ikea.com/sa/en/images/products/ekedalen-bar-table-white__0657888_pe710020_s5.jpg?f=s',
 'https://www.ikea.com/sa/en/images/products/ekedalen-bar-stool-with-backrest-white-orrsta-light-grey__0657871_pe710003_s5.jpg?f=s',
 'https://www.ikea.com/sa/en/images/products/stensele-norraryd-bar-table-and-2-bar-stools-anthracite-anthracite-black__0680383_pe719836_s5.jpg?f=s',
 'https://www.ikea.com/sa/en/images/products/nordviken-bar-stool-with-backrest-black__0714185_pe729959_s5.jpg?f=s',
 'https://www.ikea.com/sa/en/images/products/glenn-bar-stool-white-chrome-plated__0452400_pe601325_s5.jpg?f=s',
 'https://www.ikea.com/sa

In [89]:
len(valid_product_id_list)

259

In [90]:
len(valid_product_image_list)

245

In [91]:
working_ikea_dataset = ikea_dataset.loc[valid_product_id_list]
working_ikea_dataset = working_ikea_dataset[~working_ikea_dataset.index.duplicated()]

In [92]:
working_ikea_dataset

,name,category,price,link,other_colors,short_description,designer,depth,height,width
item_id,,,,,,,,,,
30180504,NORBERG,Bar furniture,225.0,https://www.ikea.com/sa/en/p/norberg-wall-moun...,No,"Wall-mounted drop-leaf table, ...",Marcus Arvonen,60.0,43.0,74.0
50406465,FRANKLIN,Bar furniture,129.0,https://www.ikea.com/sa/en/p/franklin-bar-stoo...,No,"Bar stool with backrest, foldable, ...",K Hagberg/M Hagberg,44.0,95.0,50.0
397736,NORRARYD,Bar furniture,395.0,https://www.ikea.com/sa/en/p/norraryd-bar-stoo...,No,"Bar stool with backrest, 74 cm",Nike Karlsson,47.0,103.0,46.0
340759,EKEDALEN,Bar furniture,995.0,https://www.ikea.com/sa/en/p/ekedalen-bar-tabl...,No,"Bar table, 120x80 cm",Ehlén Johansson,NaN,105.0,80.0
400545,EKEDALEN,Bar furniture,345.0,https://www.ikea.com/sa/en/p/ekedalen-bar-stoo...,No,"Bar stool with backrest, 75 cm",Ehlén Johansson,51.0,114.0,43.0
...,...,...,...,...,...,...,...,...,...,...
49269983,LANDSKRONA,Sofas & armchairs,6095.0,https://www.ikea.com/sa/en/p/landskrona-5-seat...,Yes,5-seat sofa,Tord Björklund/IKEA of Sweden,NaN,78.0,360.0
228705,BRIMNES,Sofas & armchairs,1095.0,https://www.ikea.com/sa/en/p/brimnes-day-bed-f...,No,"Day-bed frame with 2 drawers, ...",K Hagberg/M Hagberg,53.0,21.0,86.0
20396260,GRÖNLID,Sofas & armchairs,170.0,https://www.ikea.com/sa/en/p/groenlid-cover-fo...,Yes,Cover for corner section,Francis Cayouette,NaN,NaN,NaN


In [93]:
working_ikea_dataset['link'] = valid_product_image_list

In [94]:
def force_show_max_width(df):
    with pd.option_context('display.max_colwidth', None, 'display.width', None):
        display(df)

In [95]:
working_ikea_dataset['category'].unique()

array(['Bar furniture', 'Beds', 'Bookcases & shelving units',
       'Cabinets & cupboards', 'Café furniture', 'Chairs',
       'Chests of drawers & drawer units', "Children's furniture",
       'Nursery furniture', 'Outdoor furniture', 'Room dividers',
       'Sideboards, buffets & console tables', 'Sofas & armchairs'],
      dtype=object)

In [96]:
category_name_id_df = pd.DataFrame({"category_id": range(1, len(working_ikea_dataset["category"].unique()) + 1)}, working_ikea_dataset["category"].unique())
category_name_id_df.index = category_name_id_df.index.rename("category_name")
category_name_id_df

,category_id
category_name,
Bar furniture,1
Beds,2
Bookcases & shelving units,3
Cabinets & cupboards,4
Café furniture,5
Chairs,6
Chests of drawers & drawer units,7
Children's furniture,8
Nursery furniture,9


In [97]:
db_username = dbconn.db_username
db_password = dbconn.db_password
db_host = dbconn.db_host
db_name = dbconn.db_name

db_connection_str = f"mysql+pymysql://{db_username}:{db_password}@{db_host}/{db_name}"
db_connection = create_engine(db_connection_str)

In [98]:
category_name_id_df.to_sql(name='product_categories', con=db_connection, if_exists='append', chunksize=4096, method=insert_on_duplicate)

In [99]:
name_mapping = json.loads(
    pd.Series(
        index=working_ikea_dataset["category"].unique(),
        data=range(1, len(working_ikea_dataset["category"].unique()) + 1),
    ).to_json()
)

working_ikea_dataset["category"] = working_ikea_dataset["category"].replace(
    name_mapping
)

In [100]:
working_ikea_dataset.index = working_ikea_dataset.index.rename('product_id')

In [101]:
working_ikea_dataset = working_ikea_dataset.rename(columns={'category': 'category_id'})

In [102]:
working_ikea_dataset = working_ikea_dataset.rename(columns={'link': 'image'})

In [103]:
working_ikea_dataset['other_colors'] = working_ikea_dataset['other_colors'].replace({'Yes': True, 'No': False})

In [104]:
force_show_max_width(working_ikea_dataset)

,name,category_id,price,image,other_colors,short_description,designer,depth,height,width
product_id,,,,,,,,,,
30180504,NORBERG,1,225.0,https://www.ikea.com/sa/en/images/products/norberg-wall-mounted-drop-leaf-table-white__0736622_pe740674_s5.jpg?f=s,False,"Wall-mounted drop-leaf table, 74x60 cm",Marcus Arvonen,60.0,43.0,74.0
50406465,FRANKLIN,1,129.0,https://www.ikea.com/sa/en/images/products/franklin-bar-stool-with-backrest-foldable-black-black__0727485_pe735710_s5.jpg?f=s,False,"Bar stool with backrest, foldable, 63 cm",K Hagberg/M Hagberg,44.0,95.0,50.0
397736,NORRARYD,1,395.0,https://www.ikea.com/sa/en/images/products/norraryd-bar-stool-with-backrest-black__0728070_pe736043_s5.jpg?f=s,False,"Bar stool with backrest, 74 cm",Nike Karlsson,47.0,103.0,46.0
340759,EKEDALEN,1,995.0,https://www.ikea.com/sa/en/images/products/ekedalen-bar-table-white__0657888_pe710020_s5.jpg?f=s,False,"Bar table, 120x80 cm",Ehlén Johansson,NaN,105.0,80.0
400545,EKEDALEN,1,345.0,https://www.ikea.com/sa/en/images/products/ekedalen-bar-stool-with-backrest-white-orrsta-light-grey__0657871_pe710003_s5.jpg?f=s,False,"Bar stool with backrest, 75 cm",Ehlén Johansson,51.0,114.0,43.0
...,...,...,...,...,...,...,...,...,...,...
49269983,LANDSKRONA,13,6095.0,https://www.ikea.com/sa/en/images/products/landskrona-5-seat-sofa-with-chaise-longues-gunnared-dark-grey-wood__0602496_pe680404_s5.jpg?f=s,True,5-seat sofa,Tord Björklund/IKEA of Sweden,NaN,78.0,360.0
228705,BRIMNES,13,1095.0,https://www.ikea.com/sa/en/images/products/brimnes-day-bed-frame-with-2-drawers-white__1161827_pe889528_s5.jpg?f=s,False,"Day-bed frame with 2 drawers, 80x200 cm",K Hagberg/M Hagberg,53.0,21.0,86.0
20396260,GRÖNLID,13,170.0,https://www.ikea.com/sa/en/images/products/groenlid-cover-for-corner-section-inseros-white__0577139_pe668613_s5.jpg?f=s,True,Cover for corner section,Francis Cayouette,NaN,NaN,NaN


In [105]:
working_ikea_dataset.to_sql(name='products', con=db_connection, if_exists='append', chunksize=4096, method=insert_on_duplicate)

In [106]:
working_ikea_dataset.to_json('ikea_products.json')